In [1]:
!pip install dask

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 13.9 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)


In [2]:
import pandas as pd
import json
import gzip

In [ ]:
# File path
file_path = r'D:\A_Raf\MS\research\Bahar_GNN_research\Books.json.gz'

chunk_size = 5000  # Number of lines per chunk
chunks = []

# Open and read the .gz file
with gzip.open(file_path, 'rt') as f:
    chunk = []
    for i, line in enumerate(f):
        chunk.append(json.loads(line))  # Parse each JSON object
        if (i + 1) % chunk_size == 0:
            chunks.append(pd.DataFrame(chunk))  # Convert chunk to DataFrame
            chunk = []  # Reset chunk

    # Process any remaining lines
    if chunk:
        chunks.append(pd.DataFrame(chunk))

# Combine all chunks into a single DataFrame
b_df = pd.concat(chunks, ignore_index=True)


In [13]:
import pandas as pd
import json
import gzip

file_path = r'D:\A_Raf\MS\research\Bahar_GNN_research\Books.json.gz'
chunk_size = 10000  # Reduce the chunk size
chunk_number = 0

# Open and read the .gz file
with gzip.open(file_path, 'rt') as f:
    chunk = []
    for i, line in enumerate(f):
        chunk.append(json.loads(line))  # Parse each JSON object
        if (i + 1) % chunk_size == 0:
            chunk_df = pd.DataFrame(chunk)
            chunk_df.to_csv(f'D:\\A_Raf\\MS\\research\\Bahar_GNN_research\\book_data\\chunk_{chunk_number}.csv', index=False)
            chunk = []  # Reset chunk
            chunk_number += 1

    # Process any remaining lines
    if chunk:
        chunk_df = pd.DataFrame(chunk)
        chunk_df.to_csv(f'D:\\A_Raf\\MS\\research\\Bahar_GNN_research\\book_data\\chunk_{chunk_number}.csv', index=False)

# After processing, you can read the chunks from disk when needed


In [19]:
import os
import pandas as pd
from glob import glob

# Paths
input_folder = r'D:\A_Raf\MS\research\Bahar_GNN_research\book_data'  # Replace with the folder containing your CSV files
output_folder = r'D:\A_Raf\MS\research\Bahar_GNN_research\book_data_filtered'  # Replace with the folder to save filtered files

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Get all CSV files in the folder
csv_files = glob(os.path.join(input_folder, "*.csv"))

# Process each file
for file_path in csv_files:
    # Load the entire CSV file
    df = pd.read_csv(file_path)
    
    # Check if the 'overall' column exists
    if 'overall' in df.columns:
        # Filter rows where 'overall' == 5.0
        filtered_df = df[df['overall'] == 5.0]
        
        # Save to output folder with the same filename
        output_file_path = os.path.join(output_folder, os.path.basename(file_path))
        filtered_df.to_csv(output_file_path, index=False)
        
    #     print(f"Filtered and saved: {os.path.basename(file_path)}")
    # else:
    #     print(f"Skipped (no 'overall' column): {os.path.basename(file_path)}")


In [21]:
import pandas as pd
import json
import gzip

# File path
file_path = r'D:\A_Raf\MS\research\Bahar_GNN_research\meta_Books.json.gz'

# Open and read the .gz file, then load all lines
with gzip.open(file_path, 'rt') as f:
    data = [json.loads(line) for line in f]  # Parse all JSON objects at once

# Convert the list of dictionaries into a DataFrame
b_meta_df = pd.DataFrame(data)

# Display the resulting DataFrame
# print(b_meta_df.head())


In [25]:
file_path = r'D:\A_Raf\MS\research\Bahar_GNN_research\meta_book_data.csv'  # Ensure a valid file name
b_meta_df.to_csv(file_path, index=False)  # Save the DataFrame as CSV
print(f"DataFrame saved to {file_path}")


DataFrame saved to D:\A_Raf\MS\research\Bahar_GNN_research\meta_book_data.csv


In [ ]:
b_meta_df["asin"].unique()

'0000092878'

In [37]:
import os
import pandas as pd

# Paths
source_folder = r'D:\A_Raf\MS\research\Bahar_GNN_research\book_data_filtered'  # Folder with chunk CSVs
output_folder = r'D:\A_Raf\MS\research\Bahar_GNN_research\Book_data_selected_columns'  # Output folder

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# List of columns to keep
columns_to_keep = ['asin', 'reviewTime', 'overall', 'reviewText', 'reviewerID']  # Example columns, update as needed

# Loop through all CSV files in the source folder
for entry in os.scandir(source_folder):
    if entry.is_file() and entry.name.endswith('.csv'):
        # Full path of the current chunk file
        chunk_file_path = entry.path
        
        # Load the chunk DataFrame
        chunk_df = pd.read_csv(chunk_file_path)
        
        # Select only the columns you want to keep
        chunk_df_selected = chunk_df[columns_to_keep]
        
        # Save the selected columns to the output folder
        output_path = os.path.join(output_folder, entry.name)
        chunk_df_selected.to_csv(output_path, index=False)

print("Processing complete. Files with selected columns saved in the output folder.")


Processing complete. Files with selected columns saved in the output folder.


In [36]:
import os
import pandas as pd

# Paths
source_folder = r'D:\A_Raf\MS\research\Bahar_GNN_research\book_data_filtered'  # Folder with chunk CSVs
reference_file = r'D:\A_Raf\MS\research\Bahar_GNN_research\meta_book_data.csv'  # Reference CSV
output_folder = r'D:\A_Raf\MS\research\Bahar_GNN_research\Book_data_asin'  # Output folder

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load the reference file
reference_df = pd.read_csv(reference_file)

# Loop through all chunk files in the source folder
for entry in os.scandir(source_folder):
    if entry.is_file() and entry.name.endswith('.csv') and entry.name.startswith('chunk_'):
        # Full path of the current chunk file
        chunk_file_path = entry.path
        
        # Load the chunk DataFrame
        chunk_df = pd.read_csv(chunk_file_path)
        
        # Merge with reference data on 'asin'
        merged_df = chunk_df.merge(reference_df, on='asin', how='inner')  # 'inner' keeps only matching rows
        
        # Save the merged data only if there are matching rows
        if not merged_df.empty:
            output_path = os.path.join(output_folder, entry.name)
            merged_df.to_csv(output_path, index=False)
            # print(f"Saved merged rows to: {output_path}")

print("Processing complete. Merged files saved in the output folder.")


C:\Users\rafey\AppData\Local\Temp\ipykernel_6532\922031576.py:13: DtypeWarning: Columns (1,3,12,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  reference_df = pd.read_csv(reference_file)


ValueError: You are trying to merge on int64 and object columns for key 'asin'. If you wish to proceed you should use pd.concat

In [11]:
import gzip
import json
import pandas as pd

# File path
file_path = r'D:\A_Raf\MS\research\Bahar_GNN_research\meta_Books.json.gz'

# Initialize an empty list to hold the first 20 items
first_20_items = []

# Open and read the .gz file
with gzip.open(file_path, 'rt') as f:
    for i, line in enumerate(f):
        # Parse each JSON object and append it to the list
        first_20_items.append(json.loads(line))
        if i == 19:  # Stop after 20 items
            break

# Convert the list of first 20 items to a DataFrame
df_first_20 = pd.DataFrame(first_20_items)


In [12]:
df_first_20

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[It is a biology book with God&apos;s perspect...,,Biology Gods Living Creation Third Edition 10 ...,"[0669009075, B000K2P5SA, B00MD4G2N0, B000ASIPT...",,Keith Graham,[],"1,349,781 in Books (","[0019777701, B000AUCX7I, B000K2P5SA, B001CK63X...",Books,,,$39.94,0000092878,[],[]
1,"[Books, New, Used & Rental Textbooks, Medicine...",,[],,Mksap 16 Audio Companion: Medical Knowledge Se...,[],,Acp,[],"1,702,625 in Books (","[B01MUCYEV7, B01KUGTY6O]",Books,,,,000047715X,[],[]
2,"[Books, Arts & Photography, Music]",,"[Discography of American Punk, Hardcore, and P...",,"Flex! Discography of North American Punk, Hard...",[],,Burkhard Jarisch,[],"6,291,012 in Books (",[],Books,,,$199.99,0000004545,[],[]
3,"[Books, Arts & Photography, Music]",,[This is a collection of classic gospel hymns ...,,Heavenly Highway Hymns: Shaped-Note Hymnal,[],,Stamps/Baxter,[],"2,384,057 in Books (","[0006180116, 0996092730, B000QFOGY0, B06WWKNDL...",Books,,,,0000013765,[],[]
4,[],,[],,Georgina Goodman Nelson Womens Size 8.5 Purple...,[],,,[],"11,735,726 in Books (",[],Books,,,$164.10,0000000116,[],[]
5,"[Books, New, Used & Rental Textbooks, Medicine...",,[Brand new; never used.],,Principles of Analgesic Use in the Treatment o...,"[0323056962, 0123979285]",,American Pain Society,[],"2,906,939 in Books (","[0323056962, 0521879272]",Books,,,,0000555010,[],[]
6,"[Books, Medical Books, Medicine]",,[Flash cards used with accompany MKSAP 15 audi...,,MKSAP 15 Audio Companion,[],,ACP,[],"2,236,549 in Books (",[],Books,,,,0000477141,[],[]
7,"[Books, New, Used & Rental Textbooks, Business...",,[Simple Truths of Service: Inspired by Jonny t...,,The Simple Truths of Service: Inspired by John...,"[1492630519, 0071819045, 0688123163, 160810640...",,Visit Amazon's Ken Blanchard Page,[],"2,566,783 in Books (","[0692842004, 1492630519, 1978489552, 160810640...",Books,,,,0000230022,[],[]
8,"[Books, Education & Teaching, Schools & Teaching]",,"[This book will alert, amuse and appall you as...",,Double-Speak: From Revenue Enhancement to Term...,[],,William Lutz,[],"2,505,873 in Books (","[0060171340, 0060161345, 0062734121]",Books,,,$198.70,0000038504,[],[]
9,[],,[],,LJ Classique Interchangeable Ladies Gift Set W...,[],,,[],"4,368,310 in Books (",[],Books,,,,0000001589,[],[]


In [2]:
print(user_movie_data[0:5000])

{"overall": 5.0, "verified": true, "reviewTime": "03 11, 2013", "reviewerID": "A3478QRKQDOPQ2", "asin": "0001527665", "style": {"Format:": " VHS Tape"}, "reviewerName": "jacki", "reviewText": "really happy they got evangelised .. spoiler alert==happy ending liked that..since started bit worrisome... but yeah great stories these missionary movies, really short only half hour but still great", "summary": "great", "unixReviewTime": 1362960000}
{"overall": 5.0, "vote": "3", "verified": true, "reviewTime": "02 18, 2013", "reviewerID": "A2VHSG6TZHU1OB", "asin": "0001527665", "style": {"Format:": " Amazon Video"}, "reviewerName": "Ken P", "reviewText": "Having lived in West New Guinea (Papua) during the time period covered in this video, it is realistic, accurate, and conveys well the entrance of light and truth into a culture that was for centuries dead to and alienated from God.", "summary": "Realistic and Accurate", "unixReviewTime": 1361145600}
{"overall": 5.0, "verified": false, "reviewT

In [ ]:
print("22")

In [ ]:
m_user_data_lines = user_movie_data.strip().split('\n')
m_user_data_dicts = [json.loads(line) for line in m_user_data_lines]
m_user_df = pd.DataFrame(m_user_data_dicts)